## Tópico Complementarar Módulo 13: Regularização

In [60]:
# typing
from typing import Any

# EAD
import pandas as pd
import numpy as np

# machine learning
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Lasso

Nesta aula prática, vamos aplicar a regularização L1 e L2 em um modelo de regressão e avaliar sua performance. Começaremos criando um novo modelo e importando as bibliotecas necessárias. Em seguida, carregaremos os dados e faremos a separação entre treino e teste. Para aplicar a regularização, utilizaremos o modelo de regressão linear múltipla com o L1. Definiremos o hiperparâmetro alfa para controlar a intensidade da regularização. Em seguida, criaremos uma função para mostrar a importância das features com base nos coeficientes. Veremos que o L1 zerou alguns coeficientes, indicando que essas features não são relevantes para o modelo. Também avaliaremos a performance do modelo com o L1. Por fim, discutiremos a variação do L1 com validação cruzada.

In [61]:
# carregar os dados - regressão
df_lasso = pd.read_csv('../../datasets/dataset_colesterol.csv')

# visualizar estrutura de dados
df_lasso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     996 non-null    object 
 2   Fumante             997 non-null    object 
 3   Nível de Atividade  996 non-null    object 
 4   Idade               997 non-null    float64
 5   Peso                997 non-null    float64
 6   Altura              997 non-null    float64
 7   Colesterol          1000 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 62.6+ KB


In [62]:
# coletar medidas de variáveis categóricas
moda_grupo_sanguineo = df_lasso["Grupo Sanguíneo"].mode()
moda_fumante = df_lasso["Fumante"].mode()
moda_nivel_atividade_fisica = df_lasso["Nível de Atividade"].mode(
)

# coletar medidade de variáveis numéricas
mediana_idade = df_lasso["Idade"].median()
mediana_peso = df_lasso["Peso"].median()
mediana_altura = df_lasso["Altura"].median()

# imputar valores ausentes
df_lasso.fillna(
    value={
        "Grupo Sanguíneo": moda_grupo_sanguineo[0],
        "Fumante": moda_fumante[0],
        "Nível de Atividade": moda_nivel_atividade_fisica[0],
        "Idade": mediana_idade,
        "Peso": mediana_peso,
        "Altura": mediana_altura,
    },
    inplace=True,
)

In [63]:
# detectar quantidade valores ausentes (nulos) por coluna
df_lasso.isna().sum()

Id                    0
Grupo Sanguíneo       0
Fumante               0
Nível de Atividade    0
Idade                 0
Peso                  0
Altura                0
Colesterol            0
dtype: int64

In [64]:
# ajustar dataframe
df_lasso.drop("Id", axis=1, inplace=True)

# aplicar OneHotEnconding nas variáveis categóricas
df_lasso = pd.get_dummies(
    df_lasso, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

# exibir dataframe atualizado
df_lasso

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33.0,85.1,186.0,199.63,0,0,1,0,0,1,0,1,0
1,68.0,105.0,184.0,236.98,1,0,0,0,1,0,0,0,1
2,25.0,64.8,180.0,161.79,0,0,0,1,1,0,1,0,0
3,43.0,120.2,167.0,336.24,1,0,0,0,1,0,1,0,0
4,79.0,88.5,175.0,226.23,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31.0,68.1,166.0,206.81,0,1,0,0,0,1,0,0,1
996,51.0,47.7,170.0,128.03,0,0,0,1,1,0,1,0,0
997,39.0,85.5,176.0,211.14,0,1,0,0,1,0,0,1,0
998,61.0,91.2,161.0,284.53,0,1,0,0,0,1,0,1,0


In [65]:
# separar x e y
X = df_lasso.drop('Colesterol', axis=1)
y = df_lasso['Colesterol']

# treinar modelo de regressão com Lasso (L1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [66]:
# Treinar modelo de regressão Linear Múltipla com Lasso
# Quanto maior o alpha, maior a penalização e mais coeficiente tendem a ser reduzidos a zero
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [67]:
# exibir importância das features
def features_importance(model) -> Any:
    importance = np.abs(model.coef_)
    print("Importância das Features:")
    for coefs, feature, in enumerate(model.feature_names_in_):
        print(f"{feature}: {importance[coefs]}")
        
features_importance(model_lasso)

Importância das Features:
Idade: 0.021892956454610635
Peso: 2.4825937441605466
Altura: 2.196588333644872
Grupo Sanguíneo_A: 0.13835715086695366
Grupo Sanguíneo_AB: 1.0713170142143902
Grupo Sanguíneo_B: 0.5510561456671939
Grupo Sanguíneo_O: 0.02407855233079502
Fumante_Não: 1.6992076574414294
Fumante_Sim: 5.482231952353485e-15
Nível de Atividade_Alto: 1.9174226150584155
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.0


In [68]:
# avaliar performance
def performance_regression(model, X_test, y_test) -> Any:
    y_pred = model.predict(X_test)
    
    return mean_squared_error(y_test, y_pred, squared=False)

performance_regression(model_lasso, X_test, y_test)

c:\Users\Usuário\.virtualenvs\supervised_algorithms-WLhrZFFA\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.979087059280728

### Lasso CV

Na aula de hoje, aprendemos sobre a técnica de validação cruzada para executar modelos de regressão com penalização dos coeficientes. Em vez de usar o train test split, podemos usar a função LassoCV para treinar o modelo com diferentes valores de alfa. Podemos especificar os alfas desejados, a quantidade de folds e o random state para misturar os dados. Em seguida, podemos avaliar a importância das features e a performance do modelo. Também exploramos a opção de treinar o modelo com o conjunto completo de dados, em vez de apenas o conjunto de treinamento. Essa técnica de validação cruzada é uma forma eficaz de regularizar os modelos de regressão.

In [69]:
# treinar lassoCV
model_lasso_cv = LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)
model_lasso_cv.fit(X_train, y_train)

LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)

In [70]:
# importância das features LassoCV
features_importance(model_lasso_cv)

Importância das Features:
Idade: 0.021892956454610635
Peso: 2.4825937441605466
Altura: 2.196588333644872
Grupo Sanguíneo_A: 0.13835715086695366
Grupo Sanguíneo_AB: 1.0713170142143902
Grupo Sanguíneo_B: 0.5510561456671939
Grupo Sanguíneo_O: 0.02407855233079502
Fumante_Não: 1.6992076574414294
Fumante_Sim: 5.482231952353485e-15
Nível de Atividade_Alto: 1.9174226150584155
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.0


In [71]:
# avaliar performance
performance_regression(model_lasso_cv, X_test, y_test)

c:\Users\Usuário\.virtualenvs\supervised_algorithms-WLhrZFFA\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.979087059280728

### Ridge e RidgeCV

Neste vídeo, vamos explorar a regularização do tipo L2 ou Ridge. A estrutura é semelhante à regularização anterior, então podemos adaptar o modelo de treinamento para o Ridge. O Ridge não zera os coeficientes, mas aplica uma penalização na soma dos quadrados. O Alpha controla a intensidade da penalização. Vamos ajustar o modelo Ridge com um Alpha inicial de 0.1 e avaliar sua performance. O objetivo da regularização é reduzir o overfitting, mesmo que a métrica de erro não seja a melhor. Também é importante balancear a quantidade de features e controlar o overfitting. Vamos continuar com o modelo Ridge CV, que é semelhante ao Ridge, mas com validação cruzada. Avaliaremos sua performance e veremos que ele reduz a importância das features, resultando em um modelo mais simples com um erro menor. É importante encontrar o equilíbrio entre a quantidade de features, a métrica de erro e o controle do overfitting.

In [72]:
# Treinar modelo de regressão Linear Múltipla com Ridge
# Quanto maior o alpha, maior a penalização e mais coeficiente tendem a ser reduzidos a zero
model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X_train, y_train)

Ridge(alpha=0.1)

In [73]:
# importância das features
features_importance(model_ridge)

Importância das Features:
Idade: 0.019414943623067244
Peso: 2.4779075943926707
Altura: 2.1937549442867046
Grupo Sanguíneo_A: 0.5416865933578069
Grupo Sanguíneo_AB: 1.4342012357744847
Grupo Sanguíneo_B: 1.2748834662357489
Grupo Sanguíneo_O: 0.38236882380883125
Fumante_Não: 1.1026122048934737
Fumante_Sim: 1.1026122048965896
Nível de Atividade_Alto: 1.6457565178560545
Nível de Atividade_Baixo: 0.6715132252698364
Nível de Atividade_Moderado: 0.9742432925997934


In [74]:
# avaliar performance
performance_regression(model_ridge, X_test, y_test)

c:\Users\Usuário\.virtualenvs\supervised_algorithms-WLhrZFFA\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.040142506219325

In [75]:
# treinar RidgeCV
model_ridge_cv = RidgeCV(alphas=[0.1, 0.5, 1], cv=5)
model_ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=[0.1, 0.5, 1], cv=5)

In [76]:
# importância das features
features_importance(model_ridge_cv)

Importância das Features:
Idade: 0.01945802745682224
Peso: 2.4780485067155973
Altura: 2.193854143627682
Grupo Sanguíneo_A: 0.5404193084803735
Grupo Sanguíneo_AB: 1.4258862347712218
Grupo Sanguíneo_B: 1.2645131075681542
Grupo Sanguíneo_O: 0.37904618127644174
Fumante_Não: 1.0990280006220023
Fumante_Sim: 1.0990280006217787
Nível de Atividade_Alto: 1.6383239142326815
Nível de Atividade_Baixo: 0.6690203991850362
Nível de Atividade_Moderado: 0.9693035150499703


In [77]:
# avaliar performance
performance_regression(model_ridge_cv, X_test, y_test)

c:\Users\Usuário\.virtualenvs\supervised_algorithms-WLhrZFFA\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


9.039222763785668